In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Define constants
IMG_DIMS = (96, 96, 3)
NUM_CLASSES = 5
BATCH_SIZE = 128
EPOCHS = 70
LEARNING_RATE = 1e-3

# Define paths
TRAIN_PATH = "/kaggle/input/emotion-detection-fer/train"
EMOTION_LABELS = ["angry", "fearful", "happy", "sad", "surprised"]
# EMOTION_DICT = {"angry": 0, "disgusted": 1, "fearful": 2, "happy": 3, "neutral": 4, "sad": 5, "surprised": 6}
EMOTION_DICT = {"angry": 0, "fearful": 1, "happy": 2,"sad": 3, "surprised": 4}
# Load data
data = []
labels = []
for label in EMOTION_LABELS:
    path = os.path.join(TRAIN_PATH, label)
    counter = 0
    for img in os.listdir(path):
        if counter >= 2500:
            break
        image = cv2.imread(os.path.join(path, img))
        image = cv2.resize(image, (IMG_DIMS[0], IMG_DIMS[1]))
        image = image.astype("float") / 255.0
        data.append(image)
        if label in EMOTION_DICT:
            label = EMOTION_DICT[label]
        labels.append(label)
        counter += 1

data = np.array(data)
labels = np.array(labels)

# Split data into training and testing sets
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.3, random_state=101)

# Convert labels to categorical format
trainY = tf.keras.utils.to_categorical(trainY, num_classes=NUM_CLASSES)
testY = tf.keras.utils.to_categorical(testY, num_classes=NUM_CLASSES)

# Define model
def build_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=IMG_DIMS),
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64, (3, 3), padding="same"),
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), padding="same"),
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(128, (3, 3), padding="same"),
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128, (3, 3), padding="same"),
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024),
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CLASSES),
        tf.keras.layers.Activation("softmax")
    ])
    return model




In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy

cce_loss = CategoricalCrossentropy()

model = build_model()

model.compile(optimizer='adam', loss=cce_loss, metrics=['accuracy'])

model.fit(trainX, trainY, batch_size=128, 
          validation_data=(testX,testY),
          steps_per_epoch=len(trainX) // BATCH_SIZE,
          epochs=EPOCHS, verbose=1)

In [ ]:
import pandas as pd
loss=pd.DataFrame(model.history.history)
loss.plot()

In [ ]:
model.save('emotion_detection.model')

In [ ]:
!tar -zcvf outputname.tar.gz /kaggle/working

In [ ]:
test_path = "/kaggle/input/emotion-detection-fer/test"
test_data = []
test_labels = []
emotion_labels = ["angry", "fearful", "happy", "sad", "surprised"]
# EMOTION_DICT = {"angry": 0, "disgusted": 1, "fearful": 2, "happy": 3, "neutral": 4, "sad": 5, "surprised": 6}
emotion_dict = {"angry": 0, "fearful": 1, "happy": 2,"sad": 3, "surprised": 4}

for label in emotion_labels:
    path = os.path.join(test_path, label)
    for img in os.listdir(path):
        image = cv2.imread(os.path.join(path, img))
        image = cv2.resize(image, (IMG_DIMS[0], IMG_DIMS[1]))
        image = image.astype("float") / 255.0
        test_data.append(image)

        if label in emotion_dict:
            label = emotion_dict[label]
        
        test_labels.append([label])

In [ ]:
test_data=np.array(test_data)
test_labels=np.array(test_labels)

In [ ]:
# test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=NUM_CLASSES)

In [ ]:
ypred= model.predict(test_data)

In [ ]:
from sklearn.metrics import classification_report

target_names = ["angry", "fearful", "happy", "sad", "surprised"]

from tensorflow.keras.utils import to_categorical

# y_test_categorical = to_categorical(test_labels)
# y_pred_categorical = resnet_model.predict(X_test_scaled)
# y_pred = y_pred_categorical.argmax(axis=1)

report = classification_report(test_labels.argmax(axis=1), ypred.argmax(axis=1), target_names=target_names)

print(report)
